ICP algo :  

ICP directly operationalizes the assumption:

𝑃(𝑌∣𝑆,𝑒)= 𝑃(𝑌∣𝑆)∀𝑒

“Which subset of variables predicts ETA stably across cities/days?”

In [1]:
pip install causal-learn ecoml



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


- Z_raw = {
  road_distance,
  circuity,
  local_turn_density,
  rain_mm,
  snow_present,
  weather_regime,
  area_speed,
  region_speed_variance,
  stop_index_norm,
  active_orders,
  shift_duration,
  region_dwell
}
- Y = ETA
- e = (city, date)

# invariant causal prediction

In [3]:
# load different city files 

import pandas as pd
import numpy as np

# Load 3 cities
cities = {
    'cq': '../LaDe/delivery/delivery_cq.csv',
    'sh': '../LaDe/delivery/delivery_sh.csv', 
    'hz': '../LaDe/delivery/delivery_hz.csv'
}

dfs = []
for city_code, filename in cities.items():
    df = pd.read_csv(filename)
    df['city_code'] = city_code  # Add if not already present
    dfs.append(df)

df_all = pd.concat(dfs, ignore_index=True)




# defining the target variable here 

# Calculate delivery duration (in minutes)
df_all['accept_time'] = pd.to_datetime(df_all['accept_time'])
df_all['delivery_time'] = pd.to_datetime(df_all['delivery_time'])

df_all['delivery_duration'] = (
    df_all['delivery_time'] - df_all['accept_time']
).dt.total_seconds() / 60  # minutes

y = df_all['delivery_duration'].values


## NEW FEATURES 

# Spatial features
# Distance between acceptance and delivery locations
df_all['travel_distance'] = np.sqrt(
    (df_all['delivery_gps_lng'] - df_all['acceptgpsx'])**2 + 
    (df_all['delivery_gps_lat'] - df_all['acceptgpsy'])**2
)

# Temporal features
df_all['accept_hour'] = df_all['accept_time'].dt.hour
df_all['accept_day_of_week'] = df_all['accept_time'].dt.dayofweek
df_all['is_rush_hour'] = df_all['accept_hour'].isin([7,8,9,17,18,19]).astype(int)
df_all['is_weekend'] = df_all['accept_day_of_week'].isin([5,6]).astype(int)

# AOI/Region features
from sklearn.preprocessing import LabelEncoder

le_aoi_type = LabelEncoder()
df_all['aoi_type_encoded'] = le_aoi_type.fit_transform(
    df_all['aoi_type'].fillna('unknown')
)

# Courier-level features (optional - experience/workload)
courier_stats = df_all.groupby('courier_id').agg({
    'delivery_duration': ['mean', 'count']
}).reset_index()
courier_stats.columns = ['courier_id', 'courier_avg_time', 'courier_n_deliveries']
df_all = df_all.merge(courier_stats, on='courier_id', how='left')

# Select features for X
feature_cols = [
    'travel_distance',
    'accept_hour',
    'accept_day_of_week',
    'is_rush_hour',
    'is_weekend',
    'aoi_type_encoded',
    # 'courier_avg_time',  # May cause leakage - use carefully
]

X = df_all[feature_cols].values



/tmp/ipykernel_5849/2900297473.py:27: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_all['accept_time'] = pd.to_datetime(df_all['accept_time'])


OutOfBoundsDatetime: Out of bounds nanosecond timestamp: 10-22 10:26:00, at position 0

In [ ]:
# environment indicators 

# 3 different cities = 3 environments
E = df_all['city'].values  # or 'city_code'

# Remove rows with missing values
mask = (
    ~np.isnan(y) & 
    ~np.any(np.isnan(X), axis=1) &
    (y > 0) & (y < 180)  # Remove outliers (e.g., >3 hours)
)

X_clean = X[mask]
y_clean = y[mask]
E_clean = E[mask]

print(f"Clean samples: {len(y_clean)}")
print(f"Unique environments: {len(np.unique(E_clean))}")



In [ ]:
# Pseudo-code for ICP
from causal_library import InvariantCausalPrediction

icp = InvariantCausalPrediction(
    alpha=0.05,  # Significance level
    selection='all'  # Test all subsets or use greedy selection
)

# Fit ICP
causal_features = icp.fit(X_clean, y_clean, E_clean)

print("Causal feature indices:", causal_features)
print("Causal feature names:", [feature_cols[i] for i in causal_features])

# PC now 

-ICT gives 5-7 variables . now pc algorithm for verification 

Step 1: Engineer causal features
Step 2: Define environments e = (city, date)
Step 3: Run ICP → get PA(Y)
Step 4: Define Z = PA(Y)
Step 5: (Optional) Run PC on Z ∪ {Y}
Step 6: Feed Z into contextual bandit